In [1]:
from osm_flex.download import *
from osm_flex.extract import *
from osm_flex.config import *
import osm_flex.clip as cp
import geojson
import sys
from shapely.geometry import shape
from damagescanner.vector import *

from ra2ce_multi_network import deeper_extraction, simplify_rail
from simplify_rail import *
from deeper_extraction import filter_on_other_tags
import json
# if ra2ce is not found sys.path.append(r'C:path_to_the_ra2ce_repo')  

c:\Users\asgarpou\AppData\Local\miniconda3\envs\ra2ce_multi_network_env\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


ModuleNotFoundError: No module named 'deeper_extraction'

In [ ]:
### Defining ini variables
root_folder = OSM_DATA_DIR.parent

## dump-related
iso3_code = "NLD"
region_code = "Europe"

dump_region = DICT_GEOFABRIK[iso3_code][1]
# dump_region = region_code.lower()

dump_folder = root_folder / "osm_pbf"

## Clipping-related
study_area_suffix = '_ROTTERDAM_PORT'  # small case study area that works: '_ROTTERDAM_PORT'
clip_polygon_path = Path(
    rf'path_to_polygon.geojson'
)
clip_output_name = f'study_area{study_area_suffix}'
study_area_dump_path = root_folder.joinpath('osm_pbf', f'{clip_output_name}.osm.pbf')

## Extraction-related
default_osm_keys = DICT_CIS_OSM['rail']['osm_keys']
extract_path = root_folder.joinpath('extracts')

# source: https://taginfo.openstreetmap.org/tags/railway=rail#combinations
# 'other_tags' key is a string chain of 'tags' => 'keys', where relavant information is stored. e.g., whether traffic mode is freight or mixed
rail_track_attributes = {
    'osm_keys': [
        'railway', 'name', 'gauge', 'electrified', 'voltage', 'bridge', 'maxspeed', 'service', 'tunnel', 'other_tags'
    ],
    'other_tags': ['"railway:traffic_mode"=>', '"usage"=>']
}

station_attributes = {
    'osm_keys': ['railway', 'name', 'other_tags'],
    'other_tags': ['"train"=>']
}
rail_track_osm_query = """railway='rail' or railway='light_rail'"""
station_osm_query = """railway='station'"""

In [ ]:
## Get the country or region dump
# get_country_geofabrik(iso3=country_iso3, save_path=root_folder)

In [ ]:
## Clip to the study area

with open(clip_polygon_path) as f:
    clip_polygon_gj = geojson.load(f)
    
clip_polygon_gdf = gpd.read_file(clip_polygon_path)

polygon_feature = clip_polygon_gj['features'][0]
polygon_geom = shape(polygon_feature['geometry'])

# cp.clip_from_shapes([polygon_geom],
#                     osmpbf_output=OSM_DATA_DIR.joinpath(f'{clip_output_name}.osm.pbf'),
#                     osmpbf_clip_from=OSM_DATA_DIR.joinpath(f'{dump_region}-latest.osm.pbf'),
#                     kernel='osmconvert', overwrite=True)
clip_polygon_gdf.explore(tiles="CartoDB positron")

In [ ]:
## Extract required system components
# railway_system_gdf = extract_cis(study_area_dump_path, 'rail')
raw_rail_track_gdf = extract(osm_path=study_area_dump_path, geo_type='lines',
                             osm_keys=rail_track_attributes['osm_keys'], osm_query=rail_track_osm_query)

rail_track_gdf = filter_on_other_tags(
    attributes=rail_track_attributes, other_tags_keys=rail_track_attributes['other_tags'], gdf=raw_rail_track_gdf
)

raw_station_gdf = extract(osm_path=study_area_dump_path, geo_type='points',
                          osm_keys=station_attributes['osm_keys'], osm_query=station_osm_query)

station_gdf = filter_on_other_tags(
    attributes=station_attributes, other_tags_keys=station_attributes['other_tags'], gdf=raw_station_gdf,
    dropna=["train"]
)

raw_station_file = root_folder.joinpath(f'raw_station_{clip_output_name}.geojson')
raw_rail_track_file = root_folder.joinpath(f'raw_rail_track_{clip_output_name}.geojson')
rail_track_file = root_folder.joinpath(f'rail_track_{clip_output_name}.geojson')
station_file = root_folder.joinpath(f'station_{clip_output_name}.geojson')

raw_station_gdf.to_file(raw_station_file, driver='GeoJSON')
raw_rail_track_gdf.to_file(raw_rail_track_file, driver='GeoJSON')
rail_track_gdf.to_file(rail_track_file, driver='GeoJSON')
station_gdf.to_file(station_file, driver='GeoJSON')

## Alternative extraction package and method: Use damagescanner to retrieve railway
# all_rail = retrieve(osm_path=str(study_area_dump_path), geo_type='lines',
#                     key_col=rail_osm_keys, )
# rail_track_gdf = all_rail[all_rail['railway'] == 'rail']
# rail_map = rail_track_gdf.explore()
# rail_map


In [ ]:
# Visualise all railway network components
# railway_system_gdf.explore(tiles="CartoDB positron")

In [ ]:
## Visualise railway tracks
rail_map = rail_track_gdf.explore(tiles="CartoDB positron")
rail_map

In [ ]:
## Visualise railway passenger stations
station_gdf.explore(m=rail_map, color='red', marker_kwds={'radius':5})